In [17]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer
import h5py
import numpy
import tensorflow as tf
import random
import json
import spacy
from nltk.stem.porter import *

class Chatbot:

    showWorking = False
    train_again=False
    
    vocabulary=[]
    labels=[]
    responseDictionary = {}
    stemmer = PorterStemmer()
    
    def __init__(self,train_again=False):
        Chatbot.train_again = train_again
        
    spacy_nlp = spacy.load('en_core_web_sm')
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    customize_stop_words = ['.', '?','!']
    for w in customize_stop_words:
        spacy_nlp.vocab[w].is_stop = True

    def get_model(self):

        if not Chatbot.train_again:
            with open ('LabelList.json') as fl:
                Chatbot.labels = json.load(fl)
            
            with open ('VocabularyList.json') as fv:
                Chatbot.vocabulary = json.load(fv)
                
            with open ('responseDictionary.json') as fr:
                Chatbot.responseDictionary = json.load(fr)
                
            model= tf.keras.models.load_model("chatBotold.h5")
        else:
            with open('CommonIntentions.json') as file:
                data = json.load(file)

            docs_x = []
            docs_y = []

            for intent in data['intents']:
                
                Chatbot.responseDictionary[intent['tag']] = []
                responseList=[]
                for response in intent['responses']:
                    responseList.append(response)
                    
                Chatbot.responseDictionary[intent['tag']] = responseList
                
                for pattern in intent['patterns']:
                    wrds = remove_stop_words_get_lemmas(pattern)
                    if len(wrds)>0:
                        Chatbot.vocabulary.extend(wrds)
                        docs_x.append(wrds)
                        docs_y.append(intent['tag'])               

                if intent['tag'] not in Chatbot.labels:
                    Chatbot.labels.append(intent['tag'])
                    
            with open('responseDictionary.json', 'w') as fp:
                json.dump(Chatbot.responseDictionary, fp)
                
            Chatbot.vocabulary = sorted(list(set(Chatbot.vocabulary)))
            Chatbot.labels = sorted(Chatbot.labels)
            training = []
            output = []
            out_empty = [0 for _ in range(len(Chatbot.labels))]

            for x, doc in enumerate(docs_x):
                bag = []

                for w in Chatbot.vocabulary:
                    if w in doc:
                        bag.append(1)
                    else:
                        bag.append(0)

                output_row = out_empty[:]
                output_row[Chatbot.labels.index(docs_y[x])] = 1

                training.append(bag)
                output.append(output_row)

            training = numpy.array(training)
            output = numpy.array(output)


            model = tf.keras.Sequential()
            for layer in range(3):
            # Adds a densely-connected layer with specified units to the model:
                model.add(tf.keras.layers.Dense(10, activation='relu'))

        # Add a sigmoid layer with 1 output units:
            model.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))
            model.compile(optimizer='adam',loss='categorical_crossentropy',metric=['accuracy'])
            model.fit(training, output, epochs=300)
            model.save("chatBotold.h5")
            
            with open('LabelList.json', 'w') as fl:
                json.dump(Chatbot.labels, fl)
            with open('VocabularyList.json', 'w') as fv:
                json.dump(Chatbot.vocabulary, fv)
        return model

    def get_yes_list(self):
        return ['yes','sure','ok','okay','for sure','sure','totally','yep','ya','yeah','yup','certainly',
                         'definitely','of course','gladly','absolutely','indeed']
    
    def chat(self):
        
        print("Do you want to see internal processing steps:")
        choice = input("Enter you choice : ")
        if choice.lower() in self.get_yes_list():
            Chatbot.showWorking=True
        model = self.get_model()
        print("Start talking with the bot (type quit to stop)!")
        while True:
            inp = input("You: ")
            if inp.lower() == "quit":
                break
            results = model.predict(bag_of_words(inp))
            results_index = numpy.argmax(results)
            tag = Chatbot.labels[results_index]
            if Chatbot.showWorking:
                print("Highest intent proability:",results[0][results_index])
                print("Intent of sentence : ",tag)
            responses = Chatbot.responseDictionary[tag]
            print("chatbot:",random.choice(responses))
            if tag == "quit":
                break
            
                        
            
def remove_stop_words_get_lemmas(sentence):
        #doc = Chatbot.spacy_nlp(sentence)
        #lemmaList = [token.lemma_ for token in doc if not token.is_stop]
        s_words = nltk.word_tokenize(sentence)
        s_words = [Chatbot.stemmer.stem(word.lower()) for word in s_words]
        if Chatbot.showWorking:
            #print('Lemma :',lemmaList)
            print('Stems :',s_words)
        return s_words

def bag_of_words(sentence):
    bags = []
    #creating hot vector
    bag = [0 for _ in range(len(Chatbot.vocabulary))] 
    s_words = remove_stop_words_get_lemmas(sentence)

    for se in s_words:
        for i, w in enumerate(Chatbot.vocabulary):
            if w == se:
                bag[i] = 1
                
    if Chatbot.showWorking:
        print('Vector Representation of input sentence is:',bag)
            
    bags.append(bag)
    bags = numpy.array(bags)
    return bags
        
def main():
    chatbot = Chatbot(False)
    chatbot.chat()

if __name__ == '__main__':
    main()
    


Do you want to see internal processing steps:
Enter you choice : yes
Start talking with the bot (type quit to stop)!
You: weather
Stems : ['weather']
Vector Representation of input sentence is: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
import spacy
from importlib import reload 
reload(spacy)
nlp = spacy.load('en_core_web_sm')

doc = nlp(u"Apples and oranges are similar. Boots and hippos aren't.")
for token in doc:
    print(token, token.lemma_)

In [ ]:
import spacy

while (True):

    doc = input("Enter :")

    if doc=='quit':
        break
    spacy_nlp = spacy.load('en_core_web_sm')
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    doc = spacy_nlp(doc)
    customize_stop_words = [
        '.', '?','!'
    ]
    
    for w in customize_stop_words:
        spacy_nlp.vocab[w].is_stop = True

    tokens = [token.lemma_ for token in doc if not token.is_stop]

    print(tokens)




In [ ]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
print(spacy.lang.en.stop_words.STOP_WORDS)

In [ ]:
def bag_of_words(s, words):
    bags = []
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    #print('tokenizer:',s_words)
    
    print('lemmarizer:',get_lemmas(s))
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    bags.append(bag)
    bags = numpy.array(bags)
    #return numpy.array(bag)
    return bags

In [ ]:
import pickle

    
with open ('responseDictionary', 'rb') as fp:
    itemlist2 = pickle.load(fp)
    
print(itemlist2.keys())

In [ ]:
import nltk 
#from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()
sentence = 'what are you'
s_words = nltk.word_tokenize(sentence)
s_words = [stemmer.stem(word.lower()) for word in s_words]

print(s_words)


    


In [ ]:
dict = {'asds': 'Geeks', 'b': 'For', 'c': 'geeks'} 

def getList(dict): 
      
    return [*dict] 

print(dict['dhjjh']) 